In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticToolsParser
import pandas as pd

llm = ChatOpenAI(model="gpt-4o", temperature=0.4)

class ParaphrasedQuery(BaseModel):
    """You have performed query expansion to generate a paraphrasing of a question."""

    paraphrased_query: str = Field(
        ...,
        description="A unique paraphrasing of the original question.",
    )

template = """
You are worker in bank domain and your role is to generate syntetic data to train our models.
You need to rewrite the question "Вопрос"  in a different way but you need to keep the original meaning.
You need to generate at least 5 different versions of the question.
You will see question and answer, as response provide only rephrased questions, do not change answers.
Here is the original question and the answer:
{example}


Start generating the paraphrased questions:"""

prompt = PromptTemplate.from_template(template)
llm_with_tools = llm.bind_tools([ParaphrasedQuery])
syntetitc_data_generator = prompt | llm_with_tools# | PydanticToolsParser(tools=[ParaphrasedQuery])

if __name__ == '__main__':
    dataset = pd.read_csv('webitel_dataset.csv')
    print(dataset)
    for i in range(len(dataset)):
        example = dataset.iloc[i][0]
        if example.count('Вопрос') > 1:
            #for now we skip samples where we have more than one question
            pass
        else:
            res = syntetitc_data_generator.invoke({"example": example})
        
        print(res)
        break
        # print("Example:", example)
        # response = syntetitc_data_generator({"example": example})
        # print("Response:", response)
        # print("\n"*3

                                       question_answer
0    Вопрос: -Где деньги? Ответ :Денежные средства ...
1    Вопрос : Когда я получу займ, который запрашив...
2    Вопрос :Когда я получу запланированные выплаты...
3    Вопрос :Когда мои деньги будут зачислены на мо...
4    Вопрос :Почему долго поступают? Ответ :Денежны...
..                                                 ...
107  Вопрос: [НОМЕР ТЕЛЕФОНА]\tОтвет: -Спасибо, ожи...
108  Вопрос : -Спасибо\tОтвет :С вами свзяался спец...
109  "Вопрос : А как работает webitel? Ответ: Webit...
110  "Вопрос : -Почему у Вас такая большая комиссия...
111  "Вопрос : Что такое Webitel Ответ: Webitel - э...

[112 rows x 1 columns]


/tmp/ipykernel_1256949/4268214792.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  example = dataset.iloc[i][0]


content='' additional_kwargs={'tool_calls': [{'id': 'call_TUc0hr5SHKgedNfLIgeV7t4j', 'function': {'arguments': '{"paraphrased_query": "Где находятся мои деньги?"}', 'name': 'ParaphrasedQuery'}, 'type': 'function'}, {'id': 'call_yMenOEqOJEklBXOX1xcQeSxZ', 'function': {'arguments': '{"paraphrased_query": "Когда поступят деньги?"}', 'name': 'ParaphrasedQuery'}, 'type': 'function'}, {'id': 'call_Qt5hYk7Yr3DFB20OnoQpsk8k', 'function': {'arguments': '{"paraphrased_query": "Когда я получу свои деньги?"}', 'name': 'ParaphrasedQuery'}, 'type': 'function'}, {'id': 'call_GZdUKQoXsm5UKoCmA4saJKUA', 'function': {'arguments': '{"paraphrased_query": "Когда деньги будут зачислены?"}', 'name': 'ParaphrasedQuery'}, 'type': 'function'}, {'id': 'call_859UUP6gPL8wztemP57iK7QD', 'function': {'arguments': '{"paraphrased_query": "Когда ожидать поступления денег?"}', 'name': 'ParaphrasedQuery'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 153, 'prompt_tokens': 218, 'total_toke

In [16]:
import json

for tool_call in res.additional_kwargs['tool_calls']:
    question = json.loads(tool_call['function']['arguments'])['paraphrased_query']
    print(question)

Где находятся мои деньги?
Когда поступят деньги?
Когда я получу свои деньги?
Когда деньги будут зачислены?
Когда ожидать поступления денег?
